In [21]:
import h5py
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [22]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
def getdatasetname(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split("/")[-1]
    temp = filename_without_dir.split("_")[:-1]
    datasetname = "_".join(temp)
    return datasetname

In [24]:
def get_label(filename):
    if "rest" in filename:
        label = 0
    elif "math" in filename:
        label = 1
    elif "memory" in filename:
        label = 2
    elif "motor" in filename:
        label = 3
    return label

In [25]:
def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)

    return dataset, labels

In [26]:
def get_all_matrices(dir_path):
    dataset = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = dir_path + filename
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]
                dataset.append(matrix)
                labels.append(label)

    return dataset, labels

In [27]:
def scale(matrix):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(matrix)
    scaled_data = scaler.transform(matrix)
    return scaled_data

In [28]:
def downsample(dataset, frequency):
    downsampled_dataset = []

    for i in range(0, dataset.shape[1], 2034):
        second = dataset[:, i : i + 2034]
        subsample = []

        for j in range(0, 2034, int(2034 / frequency)):
            if j < second.shape[1]:
                measurement = second[:, j]
                subsample.append(measurement)

        downsampled_dataset.extend(subsample)

    return np.array(downsampled_dataset).T

### Data Pipeline

1. Get dataset
2. Downsample data
3. Scale Data
4. Train Model


Model setup


In [29]:
# TensorFlow and tf.keras
import tensorflow as tf

print(tf.__version__)

2.15.0


In [30]:
model = tf.keras.Sequential(
    [

        tf.keras.layers.Dense(
            128, activation="relu"
        ),  # The layer has 128 dense neurons using relu for activation

        # tf.keras.layers.Dense(4, activation='relu'), # The layer has 128 dense neurons using relu for activation

        tf.keras.layers.Dense(
            128, activation="relu"
        ),  # The layer has 128 dense neurons using relu for activation

        tf.keras.layers.Dense(
            4
        ),  # The final layer has 4 neurons, each representing a class for the action being done

    ]
)

In [31]:
model.compile(
    optimizer="adam",

    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),

    metrics=["accuracy"],
)

Model training pseudo:

```
For each file in the folder:
    Get action label from filename
    Get matrix
    Downsample
    Scale
    Train Model
```


In [32]:
intra_train_path = "your path here"

intra_test_path = "your path here"

In [33]:
# def train_model(dir_path):
#     dataset = []
#     labels = []
#     for filename in os.listdir(dir_path):
#         if filename.endswith(".h5"):
#             filename_path = dir_path + filename
#             with h5py.File(filename_path, "r") as f:
#                 dataset_name = getdatasetname(filename_path)
#                 label = get_label(filename)
#                 matrix = f.get(dataset_name)[()]

#                 train_meg = downsample(matrix, 113)
#                 train_meg = scale(train_meg)
#                 train_meg = np.array(train_meg.flatten())

#                 train_label = label

#                 model.fit(train_meg, train_label, epochs=10)

In [34]:
def load_data(dir_path):
    data = []
    labels = []

    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = os.path.join(dir_path, filename)
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]

                train_meg = downsample(matrix, 113)
                train_meg = scale(train_meg)
                flattened_meg = np.array(train_meg.flatten())

                data.append(flattened_meg)
                labels.append(label)

    return np.array(data), np.array(labels)

In [35]:
def train_model(dir_path, model):
    train_data, train_labels = load_data(dir_path)

    model.fit(train_data, train_labels, epochs=10)

In [36]:
def load_test_data(dir_path):
    data = []
    labels = []

    for filename in os.listdir(dir_path):
        if filename.endswith(".h5"):
            filename_path = os.path.join(dir_path, filename)
            with h5py.File(filename_path, "r") as f:
                dataset_name = getdatasetname(filename_path)
                label = get_label(filename)
                matrix = f.get(dataset_name)[()]

                matrix = downsample(matrix, 113)
                matrix = scale(matrix)
                matrix = np.array(matrix.flatten())

                data.append(matrix)
                labels.append(label)

    return np.array(data), np.array(labels)

In [37]:
def test_model(dir_path, model):
    test_data, test_labels = load_test_data(dir_path)

    test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)

    return test_loss, test_acc

In [38]:
# def test_model(dir_path):
#     dataset = []
#     labels = []
#     for filename in os.listdir(dir_path):
#         if filename.endswith(".h5"):
#             filename_path = dir_path + filename
#             with h5py.File(filename_path, "r") as f:
#                 dataset_name = getdatasetname(filename_path)
#                 label = get_label(filename)
#                 matrix = f.get(dataset_name)[()]

#                 matrix = downsample(matrix, 113)
#                 matrix = scale(matrix)

#                 dataset.append(matrix)
#                 labels.append(label)


#     datasetX = np.stack(dataset)
#     test_loss, test_acc = model.evaluate(datasetX,  labels, verbose=2)

#     return test_loss, test_acc

In [39]:
train_model(intra_train_path, model)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 1.6251 - accuracy: 0.2500
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 198.6624 - accuracy: 0.2500
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 489.7109 - accuracy: 0.2500
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 325.7228 - accuracy: 0.2500
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 219.5514 - accuracy: 0.2500
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 51.2000 - accuracy: 0.2500
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 133.7745 - accuracy: 0.2500
Epoch 8/10
1/1 [==============================] - 1s 1s/step - loss: 172.3834 - accuracy: 0.2500
Epoch 9/10
1/1 [==============================] - 1s 1s/step - loss: 166.8443 - accuracy: 0.3438
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 240.3233 - accuracy: 0.2500


In [40]:
test_loss, test_acc = test_model(intra_test_path, model)

print("\nTest accuracy:", test_acc)

1/1 - 0s - loss: 201.1091 - accuracy: 0.2500 - 214ms/epoch - 214ms/step

Test accuracy: 0.25
